In [66]:
import kagglehub
import pandas as pd
import pandera.pandas as pa

import numpy as np
from pathlib import Path


In [2]:
PATH_RAW: Path = Path('./assets/raw')


In [3]:
# # Download latest version
# path = kagglehub.dataset_download("paultimothymooney/stock-market-data")
#
# print("Path to dataset files:", path)

In [3]:
path = '/home/lipov/.cache/kagglehub/datasets/paultimothymooney/stock-market-data/versions/74'

In [4]:
!ls /home/lipov/.cache/kagglehub/datasets/paultimothymooney/stock-market-data/versions/74/stock_market_data/sp500/csv

AAL.csv    BSHI.csv   ED.csv	 HPQ.csv    MCHP.csv   PH.csv	  TIME.csv
AAP.csv    BSX.csv    EFX.csv	 HRB.csv    MCK.csv    PHM.csv	  TJX.csv
AAPL.csv   BWA.csv    EIX.csv	 HRL.csv    MCO.csv    PKG.csv	  TMO.csv
ABBV.csv   BXP.csv    EL.csv	 HSIC.csv   MDLZ.csv   PKI.csv	  TMUS.csv
ABC.csv    CAG.csv    EMN.csv	 HST.csv    MDT.csv    PLD.csv	  TRAUF.csv
ABMD.csv   CAH.csv    EMR.csv	 HSY.csv    MET.csv    PM.csv	  TROW.csv
ABT.csv    CAT.csv    ENS.csv	 HTLF.csv   MGM.csv    PNR.csv	  TRV.csv
ACN.csv    CB.csv     EOG.csv	 HUM.csv    MHK.csv    PNW.csv	  TSCO.csv
A.csv	   CCI.csv    EQIX.csv	 IBM.csv    MKTX.csv   PNWRF.csv  TSN.csv
ADI.csv    C.csv      EQR.csv	 ICE.csv    MLM.csv    PPG.csv	  TTWO.csv
ADM.csv    CDE.csv    ES.csv	 IDXX.csv   MMC.csv    PRU.csv	  TW.csv
ADP.csv    CDNS.csv   ESS.csv	 IEX.csv    MMM.csv    PSX.csv	  TWTR.csv
ADSK.csv   CF.csv     EW.csv	 IFF.csv    MNST.csv   PVH.csv	  TXN.csv
AEE.csv    CFG.csv    EXR.csv	 ILMN.csv   MO.csv     PWR.csv	  TXT.csv
AEP.c

In [80]:
from pathlib import Path

sp500_path = Path(path + '/stock_market_data/sp500/csv')
sp500_path

PosixPath('/home/lipov/.cache/kagglehub/datasets/paultimothymooney/stock-market-data/versions/74/stock_market_data/sp500/csv')

In [81]:
df_goog = pd.read_csv(sp500_path / 'GOOG.csv', parse_dates=['Date'], index_col='Date', usecols=['Date', 'Close'], dayfirst=True)

In [82]:
df_goog.head(3)

,Close
Date,
2004-08-19,2.499133
2004-08-20,2.697639
2004-08-23,2.724787


In [83]:
df_goog['Close'].pct_change()

Date
2004-08-19         NaN
2004-08-20    0.079430
2004-08-23    0.010064
2004-08-24   -0.041408
2004-08-25    0.010775
                ...   
2022-12-06   -0.025633
2022-12-07   -0.022197
2022-12-08   -0.012612
2022-12-09   -0.009367
2022-12-12   -0.006393
Name: Close, Length: 4612, dtype: float64

In [84]:
df_goog['log_return'] = np.log(df_goog["Close"] / df_goog["Close"].shift(1))
df_goog.head(3)

,Close,log_return
Date,,
2004-08-19,2.499133,NaN
2004-08-20,2.697639,0.076433
2004-08-23,2.724787,0.010013


In [85]:
df_goog = df_goog.drop(columns=['Close'])

In [86]:
df_goog = df_goog.dropna()
df_goog.head(2)

,log_return
Date,
2004-08-20,0.076433
2004-08-23,0.010013


In [93]:
df_goog.to_parquet(PATH_RAW / 'goog_return.parquet', index=True)

In [87]:
df_goog.loc['2021'].to_parquet(PATH_RAW / 'goog_return_2021.parquet', index=True)

In [88]:
csv_files = list(sp500_path.glob('*.csv'))

In [89]:
all_returns = []

In [90]:
not_in_2021 = []

In [91]:
for file in csv_files:
    ticker = file.stem

    df = pd.read_csv(
        file,
        usecols=['Date', 'Close'],
        parse_dates=['Date'],
        index_col='Date',
        dayfirst=True
    ).sort_index()

    df[ticker] = np.log(df['Close'] / df['Close'].shift(1))

    df = df[ticker]

    try:
        all_returns.append(df.loc['2021'])
    except KeyError:
        not_in_2021.append(ticker)

In [92]:
not_in_2021

['TIME']

In [51]:
df_returns = pd.concat(all_returns, axis=1).dropna(how='all')
df_returns.shape

(252, 408)

In [52]:
df_returns.shape

(252, 408)

In [18]:
df_goog.head()

,log_return
Date,
2004-08-20,0.076433
2004-08-23,0.010013
2004-08-24,-0.042289
2004-08-25,0.010717
2004-08-26,0.017859


In [19]:
df_goog.tail()

,log_return
Date,
2022-12-06,-0.025968
2022-12-07,-0.022447
2022-12-08,-0.012692
2022-12-09,-0.009411
2022-12-12,-0.006414


In [21]:
df_goog.loc['2020']

,log_return
Date,
2020-01-02,0.022446
2020-01-03,-0.004919
2020-01-06,0.024358
2020-01-07,-0.000624
2020-01-08,0.007850
...,...
2020-12-24,0.003728
2020-12-28,0.021190
2020-12-29,-0.009828


In [24]:
df_returns.loc['2021'].isna().sum()

BSHI    0
JBHT    0
EA      0
LNT     0
ALTR    0
       ..
AEE     0
ES      0
NCLH    0
AMT     0
DHI     0
Length: 409, dtype: int64

In [55]:
df_index_2021 = pd.read_csv('./assets/raw/INDEX_US_S&P_US_SPX_2021.csv', parse_dates=['Date'], index_col='Date', usecols=['Date', 'Close'], thousands=',').sort_index()

In [56]:
df_index_2021.head()

,Close
Date,
2021-01-04,3700.65
2021-01-05,3726.86
2021-01-06,3748.14
2021-01-07,3803.79
2021-01-08,3824.68


In [58]:
df_index_2021['Close'].dtype

dtype('float64')

In [59]:
df_index_2021['log_return'] = np.log(df_index_2021["Close"] / df_index_2021["Close"].shift(1))

In [60]:
df_index_2021 = df_index_2021.drop(columns=['Close']).dropna()

In [61]:
df_index_2021.head()

,log_return
Date,
2021-01-05,0.007058
2021-01-06,0.005694
2021-01-07,0.014738
2021-01-08,0.005477
2021-01-11,-0.006576


In [70]:
schema_log_returns = pa.DataFrameSchema(
    {
        'log_return': pa.Column(
            float,
            checks=[
                pa.Check.greater_than(-1),
                pa.Check.less_than(1),
            ],
            nullable=False,
        )
    },
    index=pa.Index(
        pa.DateTime,
        name='Date',
        checks=[
            pa.Check.in_range('2021-01-01', '2021-12-31'),
            pa.Check(lambda idx: idx.is_monotonic_increasing, element_wise=False)
        ]
    ),
    strict=True,
    coerce=True,

)

In [71]:
schema_log_returns.validate(df_index_2021)

,log_return
Date,
2021-01-05,0.007058
2021-01-06,0.005694
2021-01-07,0.014738
2021-01-08,0.005477
2021-01-11,-0.006576
...,...
2021-12-27,0.013744
2021-12-28,-0.001011
2021-12-29,0.001401


In [75]:
df_goog_2021 = pd.read_parquet('./assets/raw/goog_return_2021.parquet')

In [77]:
schema_log_returns.validate(df_goog_2021)

,log_return
Date,
2021-01-04,-0.013586
2021-01-05,0.007310
2021-01-06,-0.003239
2021-01-07,0.029504
2021-01-08,0.011106
...,...
2021-12-27,0.006243
2021-12-28,-0.010974
2021-12-29,0.000386


In [94]:
df_index_2021.to_parquet(PATH_RAW / 'sp500_returns_2021.parquet', index=True)